#### Instalar twilio

In [ ]:
#!pip3 install twilio

In [ ]:
#!pip3 install twilio requests pandas beautifulsoup4 tqdm

In [ ]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


#### URL API

In [ ]:
query = 'limache'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

In [ ]:
response = requests.get(url_clima).json()
response

In [ ]:
response.keys()

In [ ]:
response['forecast']['forecastday'][0].keys()

In [ ]:
len(response['forecast']['forecastday'][0]['hour'])

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]['time']

In [ ]:
# fecha 
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

In [ ]:
# hora del dia 
int (response['forecast']['forecastday'][0]['hour'][0]['time'].split()[1].split(':')[0])

In [ ]:
# condicion del clima
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

In [ ]:
# tenperatura en celsius
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

In [ ]:
# will_it_rain nos dice si llovera o no (1;0)
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

In [ ]:
# probabilidad de lluvia %
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

### Creando mi DATAFRAME

In [ ]:
# funcion para obtener todos los datos necesarios
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [ ]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))

In [ ]:
datos

In [ ]:
# DATAFRAME

col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

In [ ]:
# FILTRAMOS SOLO SI LLUEVE
#df[df['Lluvia']==1]

In [ ]:
# FILTRAMOS LAS HORAS QUE NECESITAMOS y si queremos podemos crear otro filtro para una consicion climatica en especial en este caso solo filtramos un rango de horas

df_rain =  df[(df['Hora']>10) & (df['Hora']< 19)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)
df_rain

In [ ]:
# ejemplo de mensaje que llegara al telefono (no llegara asi de desordenado, llegara formteado)
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

In [ ]:
# numero de telefono que compramos para hacer las pruebas
PHONE_NUMBER

#### ENVINDO MENSAJE SMS CON TWILIO

In [ ]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)

message = client.messages.create(
    from_='+12623846544',
    body='\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
    to='+56965747898'
)

print('Mensaje Enviado ' + message.sid)